In [ ]:
import os, numpy, pandas, datetime, time, json, requests, re, configparser
from sklearn.preprocessing import MinMaxScaler
from flask import Flask, jsonify
from flask import request
import json
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model

def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
    return numpy.array(dataX)

def load_rnn_model(filepath, dataset, count=1):
    print('load rnn')
    model = load_model(filepath)
    raw_data = dataset
    print('raw_data: {}'.format(raw_data))
    testPredict = model.predict(dataset)
    return raw_data, testPredict

app = Flask(__name__)
@app.route("/predict", methods=['POST'])
def predict():
    data_list1 = []
    data_list2 = []
    data_list3 = []
    data_list_all = []
    post_data1 = request.form.get('feature_1')
    post_data2 = request.form.get('feature_2')
    for value in post_data1.split(','):
        data_list1.append(float(value))
    for value in post_data2.split(','):
        data_list2.append(float(value))
    data_list3.append(data_list1)
    data_list3.append(data_list2)
    data_list_all.append(data_list3)
    data = numpy.array(data_list_all)
    raw_data, predict_data = load_rnn_model('/rnn_model.h5', data)
    for datas in predict_data:
        for data in datas:
            predict_data = data
    return jsonify({'message': str(predict_data)})

port = os.getenv('PORT', '7500')

if __name__ == "__main__":
    app.run(host = '0.0.0.0', port = int(port))